In [56]:
#On utilise un echantilloneur de Gibbs avec balayage aléatoire
import pandas as pd 
import numpy as np

T=pd.read_csv("/Users/y.boukhateb/Desktop/Statapp Optimal Transport/Stat-App/DATA/dataset1.csv")
T.drop(T.columns[0], axis=1, inplace=True)
N=len(T.columns)-1
T.head()


,/,North America,Central America,South America,North Africa,Sub-Saharan Africa,Northern Europe,Western Europe,Southern Europe,Eastern Europe,Central Asia,Western Asia,South Asia,East Asia,South-East Asia,Oceania
0,North America,96102,208668,68240,4253,58827,248379,279267,551959,162959,85259,131288,158,39145,41758,52303
1,Central America,3233173,215008,36975,390,2414,42572,81356,190931,4962,3145,6236,14,1754,510,7541
2,South America,394674,26456,600759,417,19431,68645,157063,1200694,18644,5090,32516,351,110426,2035,28490
3,North Africa,62193,93,692,57855,12512,22807,404765,442162,5437,1382,367718,1,312,340,6858
4,Sub-Saharan Africa,478694,793,10738,44230,3717881,483736,430954,312329,6442,2724,305291,243,2918,576,149130


In [58]:
T_matrix = T.iloc[:, 1:].values

In [59]:
import ot
sigma=0.04
def proba_rejet(u,v,W,x,y,Z):
    return np.exp((1/(2*sigma**2)*(np.linalg.norm(T_matrix-ot.emd(u,v,W))**2-np.linalg.norm(T_matrix-ot.emd(x,y,Z))**2)))

def in_hypercube(u,v,W):
    # Vérification des coefficients des vecteurs u et v
    if any(x < 0 or x > 1 for x in u):
        return False
    if any(x < 0 or x > 1 for x in v):
        return False
    
    # Vérification des coefficients de la matrice W
    if any(any(x < 0 or x > 1 for x in row) for row in W):
        return False
    
    # Si toutes les vérifications sont réussies, renvoyer True
    return True

In [113]:
# Fixer la graine aléatoire
np.random.seed(123)

# Taille de l'échantillon
n = 10**5

# Initialisation
U, V, C = np.zeros((n, N)), np.zeros((n, N)), np.zeros((n, N, N))
U[0] = np.random.uniform(size=N)
V[0] = np.random.uniform(size=N)
C[0, :, :] = np.random.uniform(size=(N, N))

# Vecteurs aléatoires
e_u = np.random.normal(0, 0.4, (N, n))
e_v = np.random.normal(0, 0.4, (N, n))
e_C = np.random.normal(0, 0.4, (N, N, n))
u_u = np.random.uniform(size=n)
u_v = np.random.uniform(size=n)
u_c = np.random.uniform(size=n)

for k in range(n-1):
    # Pas pour U
    x, y, Z = U[k] + e_u[:, k], V[k], C[k]
    if not in_hypercube(x, y, Z):
        U[k+1], V[k+1], C[k+1] = U[k], V[k], C[k]
    else:
        if u_u[k] < proba_rejet(U[k], V[k], C[k], x, y, Z):
            U[k+1], V[k+1], C[k+1] = x, y, Z
        else:
            U[k+1], V[k+1], C[k+1] = U[k], V[k], C[k]

    # Pas pour V
    x, y, Z = U[k], V[k] + e_v[:, k], C[k]
    if not in_hypercube(x, y, Z):
        U[k+1], V[k+1], C[k+1] = U[k], V[k], C[k]
    else:
        if u_v[k] < proba_rejet(U[k], V[k], C[k], x, y, Z):
            U[k+1], V[k+1], C[k+1] = x, y, Z
        else:
            U[k+1], V[k+1], C[k+1] = U[k], V[k], C[k]

    # Pas pour C
    x, y, Z = U[k], V[k], C[k] + e_C[:, :, k]
    if not in_hypercube(x, y, Z):
        U[k+1], V[k+1], C[k+1] = U[k], V[k], C[k]
    else:
        if u_c[k] < proba_rejet(U[k], V[k], C[k], x, y, Z):
            U[k+1], V[k+1], C[k+1] = x, y, Z
        else:
            U[k+1], V[k+1], C[k+1] = U[k], V[k], C[k]


In [110]:
def histogramme(x):
    # Normaliser la liste pour que la somme soit égale à 1
    somme = np.sum(x)
    histogramme = x / somme
    return histogramme

In [114]:
# Calcul de la proportion de lignes distinctes dans la matrice U
print(U)

[[0.69646919 0.28613933 0.22685145 ... 0.43857224 0.0596779  0.39804426]
 [0.69646919 0.28613933 0.22685145 ... 0.43857224 0.0596779  0.39804426]
 [0.69646919 0.28613933 0.22685145 ... 0.43857224 0.0596779  0.39804426]
 ...
 [0.69646919 0.28613933 0.22685145 ... 0.43857224 0.0596779  0.39804426]
 [0.69646919 0.28613933 0.22685145 ... 0.43857224 0.0596779  0.39804426]
 [0.69646919 0.28613933 0.22685145 ... 0.43857224 0.0596779  0.39804426]]
